# Preprocessing data to graph structure

In [116]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [117]:
%reload_kedro

2022-08-30 17:39:02,014 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-30 17:39:02,162 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-30 17:39:02,375 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-30 17:39:02,377 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-30 17:39:02,407 - root - INFO - Registered line magic `run_viz`
2022-08-30 17:39:02,408 - root - INFO - Registered line magic `reload_kedro_mlflow`


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [118]:
from typing import Iterator, Tuple
import re

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

import gid_ml_framework.pipelines.santander_preprocessing.nodes
from gid_ml_framework.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [122]:
customers_path = "santander_customers"
articles_path = "santander_articles"
transactions_train_path = "santander_transactions_train"
transactions_val_path = "santander_transactions_val"

In [123]:
customers = context.catalog.load(customers_path)
articles = context.catalog.load(articles_path)
transactions_train = _concat_chunks(context.catalog.load(transactions_train_path))
transactions_val = _concat_chunks(context.catalog.load(transactions_val_path))

2022-08-30 17:39:48,684 - kedro.io.data_catalog - INFO - Loading data from `santander_customers` (CSVDataSet)...
2022-08-30 17:39:50,104 - kedro.io.data_catalog - INFO - Loading data from `santander_articles` (CSVDataSet)...
2022-08-30 17:39:50,140 - kedro.io.data_catalog - INFO - Loading data from `santander_transactions_train` (CSVDataSet)...
2022-08-30 17:39:50,197 - kedro.io.data_catalog - INFO - Loading data from `santander_transactions_val` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/ked

In [124]:
customers.shape

(93146, 44)

In [111]:
articles.shape

(24, 1)

In [113]:
transactions_train.shape

(52432, 3)

In [115]:
transactions_val.shape

(3573, 3)

In [125]:
def _create_mapping(df: pd.DataFrame, map_column: str):
    """Creates mapping into consecutive integers for given column."""
    ids = df.loc[:, map_column].sort_values().reset_index(drop=True)
    mapping = {v: k for k, v in enumerate(ids)}
    return mapping

In [127]:
users_mapping = _create_mapping(customers, map_column="customer_id")
items_mapping = _create_mapping(articles, map_column="article_id")
transactions_train.replace({"customer_id": users_mapping, "article_id": items_mapping}, inplace=True)

KeyError: 'customer_id'

In [129]:
customers

,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,F,ES,V,53,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,191298.656250,01 - TOP,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1
1,F,ES,V,48,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,163073.187500,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1
2,F,ES,V,56,1995-01-16,0,254,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,128376.242188,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,N,ES,V,61,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,141979.265625,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
4,F,ES,V,54,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,155722.015625,01 - TOP,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93141,N,ES,H,47,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MALAGA,0,95195.492188,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93142,N,ES,H,66,2016-05-31,1,0,1,UNKNOWN,P,A,S,S,UNKNOWN,UNKNOWN,N,MALAGA,0,95195.492188,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93143,N,ES,V,43,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,ASTURIAS,0,86066.429688,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93144,N,ES,V,31,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MADRID,0,138027.750000,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [86]:
transactions.sort_values("customer_id")

,customer_id,date,article_id
25159,103,2015-06-28,16
27383,103,2015-11-28,16
24424,103,2015-03-28,16
34914,103,2015-11-28,15
29437,103,2016-02-28,16
...,...,...,...
6243,1542563,2016-04-28,2
51518,1542616,2016-04-28,20
6245,1542616,2016-04-28,2
6244,1542649,2016-04-28,2


In [87]:
customers.loc[customers.customer_id==1542649, :]

,customer_id,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1


In [90]:
customers.shape[0]

92868

In [88]:
len(transactions.customer_id.unique())

19330